In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('/home/user/project/sharp_features/')

In [2]:
from sharpf.models import load_model, MODEL_BY_NAME
from sharpf.modules.base import load_with_spec
from sharpf.modules import module_by_kind
import torch
import numpy as np

import json
spec = json.load(open('/home/user/project/sharp_features/sharpf/models/specs/dgcnn.json', 'r'))

from sharpf.data.data import ABCData

data_label = 'points'
target_label = 'distances'
data_root = '/home/user/project/point_patches/'
dtst = ABCData(
    data_path=data_root, partition='train',
    data_label=data_label,
    target_label=target_label
)

12208593.0 out of 4733 items are sharp


In [3]:
from model_seg import DGCNN
from collections import namedtuple
torch.cuda.empty_cache()


device = 'cuda:2'
points = dtst[0][0]
points = torch.tensor(np.stack([points for idx in range(10)]), device=device)

artons_model = load_with_spec(spec, MODEL_BY_NAME).to(device).eval()
Args = namedtuple('Args', ['k', 'dropout'])
args = Args(10, 0.5)
dgcnn_model = DGCNN(args).to(device).eval()

art_res = artons_model(points)
dgcnn_res = dgcnn_model(points.permute(0,2,1))

In [7]:
dgcnn_model

DGCNN(
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn7): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn8): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1): Sequential(
    (0): Conv2d(6, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2

In [8]:
artons_model

DGCNN(
  (encoder_blocks): ModuleList(
    (0): PointOpBlock(
      (_op): Sequential(
        (0): NeighbourKNN()
        (1): LocalDynamicGraph()
        (2): StackedConv(
          (conv): Sequential(
            (0): Sequential(
              (0): Conv2d(6, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): ReLU(inplace=True)
            )
            (1): Sequential(
              (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): ReLU(inplace=True)
            )
          )
        )
        (3): AggregationMax()
        (4): InterpolationBase()
      )
    )
    (1): PointOpBlock(
      (_op): Sequential(
        (0): NeighbourKNN()
        (1): LocalDynamicGraph()
        (2): StackedConv(
          (conv): Sequential(
           